##  Knockout


In [1]:
#https://cobrapy.readthedocs.io/en/latest/deletions.html
from cobra.io import read_sbml_model
from cobra import Reaction, Metabolite
from cobra.flux_analysis import single_reaction_deletion, double_reaction_deletion

In [2]:
# Read model (resveratrol from glucose, S. cervisae)
model = read_sbml_model('../data/models/yeast_resv.xml')

In [18]:
model.medium

{'EX_fe2_e': 999999.0,
 'EX_glc__D_e': 10.0,
 'EX_h2o_e': 999999.0,
 'EX_h_e': 999999.0,
 'EX_k_e': 999999.0,
 'EX_na1_e': 999999.0,
 'EX_so4_e': 999999.0,
 'EX_nh4_e': 999999.0,
 'EX_o2_e': 2.0,
 'EX_pi_e': 999999.0}

In [19]:
knockout=['ANS', 'PPNDH']

In [3]:
model.reactions.ANS

Reaction identifier,ANS
Name,Anthranilate synthase
Memory address,0x01a4396d3d08
Stoichiometry,chor_c + gln__L_c --> anth_c + glu__L_c + h_c + pyr_c Chorismate + L-Glutamine --> Anthranilate + L-Glutamate + H+ + Pyruvate
GPR,YER090W and YKL211C
Lower bound,0.0
Upper bound,999999.0


In [21]:
m.reactions.ANS.knock_out()

In [22]:
m.reactions.ANS

Reaction identifier,ANS
Name,Anthranilate synthase
Memory address,0x01ffb6413c08
Stoichiometry,chor_c + gln__L_c --> anth_c + glu__L_c + h_c + pyr_c Chorismate + L-Glutamine --> Anthranilate + L-Glutamate + H+ + Pyruvate
GPR,YER090W and YKL211C
Lower bound,0
Upper bound,0


In [27]:
ko_m=model

In [40]:
print(ko_m.optimize())
with ko_m:
    ko_m.reactions.ATPM.knock_out()
    print(ko_m.optimize())
    ko_m.objective=ko_m.reactions.VVVST1
    
    rv_maxp= ko_m.optimize().objective_value
    print("Maximum resveratrol productivity = ",rv_maxp , "mmol/gDW*h")

    glucose_uptake_flux=ko_m.reactions.EX_glc__D_e.flux #Glucose exchange reaction
    max_yield = rv_maxp / (-1*glucose_uptake_flux)
    print("Maximum theoretical yield is = ", max_yield, "mmol/gDW*h")

<Solution 0.288 at 0x1ffb7f20508>
<Solution 0.300 at 0x1ffb7d20108>
Maximum resveratrol productivity =  1.2214285714285713 mmol/gDW*h
Maximum theoretical yield is =  0.12214285714285714 mmol/gDW*h


In [41]:
df1=single_reaction_deletion(model, model.reactions)
df1.sort_values(by=['growth'], ascending=False, inplace=True)
df1

,ids,growth,status
371,{ATPM},2.997446e-01,optimal
1179,{VALt2r},2.878657e-01,optimal
1177,{QULNS},2.878657e-01,optimal
1176,{METB1},2.878657e-01,optimal
1175,{ERGSTESTH_SCe},2.878657e-01,optimal
...,...,...,...
1078,{GLUPRT},-2.050005e-13,optimal
277,{PRPPS},-3.231930e-13,optimal
1273,{ADSL2r},-6.597813e-13,optimal
485,{GLCt1},NaN,infeasible


In [38]:
model.reactions.QULNS

Reaction identifier,QULNS
Name,Quinolinate synthase
Memory address,0x01ffb7e28848
Stoichiometry,dhap_c + iasp_c --> 2.0 h2o_c + pi_c + quln_c Dihydroxyacetone phosphate + Iminoaspartate --> 2.0 H2O H2O + Phosphate + Quinolinate
GPR,
Lower bound,0.0
Upper bound,999999.0


In [ ]:
df2=double_reaction_deletion(model, model.reactions)
df2.sort_values(by=['growth'], ascending=False, inplace=True)
df2

In [6]:
mo=model
print(mo.optimize())
with mo:
    single=single_reaction_deletion(mo)
    res=single.knockout['ATPM']
    print(res)
    #mo.reactions.ATPM.knock_out()
    print(mo.optimize())
    mo.objective=mo.reactions.VVVST1
    
    rv_maxp= mo.optimize().objective_value
    print("Maximum resveratrol productivity = ",rv_maxp , "mmol/gDW*h")

    glucose_uptake_flux=mo.reactions.EX_glc__D_e.flux #Glucose exchange reaction
    max_yield = rv_maxp / (-1*glucose_uptake_flux)
    print("Maximum theoretical yield is = ", max_yield, "mmol/gDW*h")

<Solution 0.288 at 0x1a439c77d08>
        ids    growth   status
941  {ATPM}  0.299745  optimal
<Solution 0.288 at 0x1a439cb1c88>
Maximum resveratrol productivity =  1.1785714285714288 mmol/gDW*h
Maximum theoretical yield is =  0.11785714285714288 mmol/gDW*h


### Try with OptKnock

In [ ]:
from cameo import models
from cameo.visualization.plotting.with_plotly import PlotlyPlotter